In [15]:
# Import libraries
import logging
import time
from bs4 import BeautifulSoup
import requests
import pandas as pd
import time
import logging
from itertools import chain

In [20]:
class Scraper(object):
    """
    Pull player info down from web
    """

    # Initialize array to store player
    players_scraped = []

    # Instantiate scraper
    def __init__(self, urls):
        self.urls = urls
        self.logger = logging.getLogger("sLogger")

    # request to get the url
    def get_page(self, url):
        response = requests.get(url)
        if response.status_code:
            soup = BeautifulSoup(response.content, "html.parser")
            return soup.find("tbody", {"class": "list"})
        else:
            self.logger.error("Error" + response.status_code)
            return None

    # helper method to get players
    def get_players(self,trs):
        return [extract_info(tr) for tr in trs]
    
    # helper method to get stats
    def get_stats(self,trs):
        return [extract_stats(tr) for tr in trs]

    # method to extract and copy player info from web
    def scrap(self, urls):
        for url in urls:
            tbody = self.get_page(url)
            if tbody is None:
                continue
            trs = tbody.findAll("tr")
            Scraper.players_scraped.append(self.get_players(trs))
            self.logger.info("Page{} scraped".format(len(Scraper.players_scraped)))
    # method to start the scraper
    def start(self):
        self.scrap(self.urls)

In [21]:
params = {"ae": "0", "oa": "1", "pt": "2", "vl": "3", "wg": "4", "bp": "5"}
query = "&".join([f"showCol%5B{y}%5D={x}" for x, y in params.items()])
url = f"https://sofifa.com/players?{query}&offset="
urls = [url + str(offset) for offset in range(0, 120, 60)]
# Parameters
number_of_scraper = 31
pages = 10

scrapers = [Scraper(urls[pages * i:min(pages * (i + 1), len(urls))]) 
            for i in range(number_of_scraper)]
multi_threading = MultiThreading(scrapers)

#     multi_threading.run()


In [111]:
t1 = time.time()
multi_threading.run()
def flatten(d):
    out = {}
    for key, val in d.items():
        if isinstance(val, dict):
            val = [val]
        if isinstance(val, list):
            for subdict in val:
                deeper = flatten(subdict).items()
                out.update({key2: val2 for key2, val2 in deeper})
        else:
            out[key] = val
    return out
players = list(map(flatten, (chain.from_iterable(Scraper.players_scraped))))

df_multi_thread = pd.DataFrame(players)
df_multi_thread.drop_duplicates(ignore_index = True)
print("Multi threading time taken: ", time.time() - t1)
df_multi_thread.head()


Multi threading time taken:  56.962055921554565


,name,country,age,overall,potential,club,best_position,value,wage,Crossing,...,Vision,Penalties,Composure,Defensive Awareness,Standing Tackle,Sliding Tackle,Diving,Handling,Kicking,Reflexes
0,Aurélien Tchouaméni,France,21,81,88,AS Monaco,CM,€52M,€46K,68,...,82,55,82,80,83,78,9,8,12,7
1,Adam Hložek,Czech Republic,18,77,87,AC Sparta Praha,CAM,€22M,€500,69,...,78,66,80,23,32,23,11,10,9,10
2,Saša Kalajdžić,Austria,23,77,82,VfB Stuttgart,ST,€16M,€27K,59,...,70,41,72,39,46,47,7,6,13,15
3,Noussair Mazraoui,Morocco,23,82,86,Ajax,RB,€40.5M,€20K,78,...,76,44,79,74,77,80,12,7,13,12
4,Rafael da Conceição Leão,Portugal,22,82,90,AC Milan,LW,€68.5M,€52K,69,...,74,63,81,22,24,21,10,12,15,9


In [114]:
df.to_csv('../data/raw/full_stats.csv')  

In [110]:
test_array = Scraper.players_scraped
players = list(chain.from_iterable(test_array))
te = list(map(flatten, players))
df = pd.DataFrame(te)
df.drop_duplicates(ignore_index = True)

,name,country,age,overall,potential,club,best_position,value,wage,Crossing,...,Vision,Penalties,Composure,Defensive Awareness,Standing Tackle,Sliding Tackle,Diving,Handling,Kicking,Reflexes
0,Aurélien Tchouaméni,France,21,81,88,AS Monaco,CM,€52M,€46K,68,...,82,55,82,80,83,78,9,8,12,7
1,Adam Hložek,Czech Republic,18,77,87,AC Sparta Praha,CAM,€22M,€500,69,...,78,66,80,23,32,23,11,10,9,10
2,Saša Kalajdžić,Austria,23,77,82,VfB Stuttgart,ST,€16M,€27K,59,...,70,41,72,39,46,47,7,6,13,15
3,Noussair Mazraoui,Morocco,23,82,86,Ajax,RB,€40.5M,€20K,78,...,76,44,79,74,77,80,12,7,13,12
4,Rafael da Conceição Leão,Portugal,22,82,90,AC Milan,LW,€68.5M,€52K,69,...,74,63,81,22,24,21,10,12,15,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
120,Dayot Upamecano,France,22,82,87,FC Bayern München,CB,€42.5M,€70K,48,...,58,39,71,78,84,90,6,7,8,9
121,Eden Hazard,Belgium,30,84,84,Real Madrid CF,LW,€40.5M,€220K,77,...,84,87,88,34,27,22,11,12,6,8
122,Bukayo Saka,England,19,82,89,Arsenal,RM,€59.5M,€58K,81,...,81,70,78,65,69,67,7,8,13,13
123,Carney Chukwuemeka,England,17,64,86,Aston Villa,CAM,€1.8M,€2K,53,...,64,58,65,43,49,54,12,12,9,10


In [19]:
# helper method for extracting data of the player

def extract_info(tr):
    base = "https://sofifa.com/"
    link = base + tr.select('td.col-name')[0].find("a").get("href")
    return {
        "name": tr.select('td.col-name')[0].find("a").get("aria-label"),
        "country": tr.select('td.col-name')[0].find("img").get("title"),
        "age": tr.select('td.col.col-ae')[0].text.strip(),
        "overall": tr.select('td.col.col-oa')[0].text.strip(),
        "potential": tr.select('td.col.col-pt')[0].text.strip(),
        "club": tr.select("td.col-name")[1].find("a").text,
        "best_position": tr.select('td.col-name')[0].find("span").text,
        "value": tr.select('td.col.col-vl')[0].text.strip(),
        "wage": tr.select('td.col.col-wg')[0].text.strip(),
        "stats" : extract_stats(link),
    }
    

# helper method for extracting stats of single player

def extract_stats(link):
    new_res = requests.get(link)
    soup = BeautifulSoup(new_res.content, "html.parser")
    tbody = soup.find_all("div", {"class":"center"})[5]
    stats_block = tbody.findAll("div", {"class":"block-quarter"})
    return extract_deep(stats_block)

def extract_deep(tr):
    return {"Att": extract_att(tr[0]) 
        , "Skill" : extract_skill(tr[1]) 
        , "Move" : extract_move(tr[2])
        , "Power" : extract_pow(tr[3])
        , "Mentality" : extract_mentality(tr[4])
        , "Defending" : extract_def(tr[5])
        , "Goalkeep" : extract_goalkeep(tr[6])} 
    
    
    

In [17]:
def extract_att(tr):
    return {"Crossing" : tr.findAll("li")[0].find("span").text.strip(),
           "Finishing" : tr.findAll("li")[1].find("span").text.strip(),
           "Heading Accuracy":  tr.findAll("li")[2].find("span").text.strip(),
           "Short passing":tr.findAll("li")[3].find("span").text.strip(),
           "Volleys": tr.findAll("li")[4].find("span").text.strip()
            }

def extract_skill(tr):
    return {"Dribbling" : tr.findAll("li")[0].find("span").text.strip(),
           "Curve" : tr.findAll("li")[1].find("span").text.strip(),
           "Fk Accuracy": tr.findAll("li")[2].find("span").text.strip(),
           "Long Passing": tr.findAll("li")[3].find("span").text.strip(),
           "Ball Control": tr.findAll("li")[4].find("span").text.strip()
            }

def extract_move(tr):
    return {"Acceleration" : tr.findAll("li")[0].find("span").text.strip(),
           "Sprint Speed" : tr.findAll("li")[1].find("span").text.strip(),
           "Agility": tr.findAll("li")[2].find("span").text.strip(),
           "Reactions": tr.findAll("li")[3].find("span").text.strip(),
           "Balance": tr.findAll("li")[4].find("span").text.strip()
            }

def extract_pow(tr):
    return {"Shot Power" : tr.findAll("li")[0].find("span").text.strip(),
           "Jumping" : tr.findAll("li")[1].find("span").text.strip(),
           "Stamina": tr.findAll("li")[2].find("span").text.strip(),
           "Strength": tr.findAll("li")[3].find("span").text.strip(),
           "Long Shots": tr.findAll("li")[4].find("span").text.strip()
            }

def extract_mentality(tr):
    return {"Aggression" : tr.findAll("li")[0].find("span").text.strip(),
           "Interceptions" : tr.findAll("li")[1].find("span").text.strip(),
           "Positioning": tr.findAll("li")[2].find("span").text.strip(),
           "Vision": tr.findAll("li")[3].find("span").text.strip(),
           "Penalties": tr.findAll("li")[4].find("span").text.strip(),
           "Composure": tr.findAll("li")[5].find("span").text.strip()
            }

def extract_def(tr):
    return {"Defensive Awareness" : tr.findAll("li")[0].find("span").text.strip(),
           "Standing Tackle" : tr.findAll("li")[1].find("span").text.strip(),
           "Sliding Tackle": tr.findAll("li")[2].find("span").text.strip()
            }

def extract_goalkeep(tr):
    return {"Diving" : tr.findAll("li")[0].find("span").text.strip(),
           "Handling" : tr.findAll("li")[1].find("span").text.strip(),
           "Kicking": tr.findAll("li")[2].find("span").text.strip(),
           "Positioning": tr.findAll("li")[3].find("span").text.strip(),
           "Reflexes": tr.findAll("li")[4].find("span").text.strip()
            }
    


## Prototype

- Refactor the Scraper class, so that it has different branch of statements to handle either scrap surface only or scrap deep only